The following code took reference from:
- https://here.isnew.info/how-to-calculate-the-area-of-a-certain-class-in-a-geotiff-file-using-numpy.html
- https://thinkinfi.com/clip-raster-with-a-shape-file-in-python/

Burn severity data:
- https://data.fs.usda.gov/geodata/rastergateway/ravg/index.php


In [24]:
import fiona
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

import geopandas as gpd

from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
def make_projections_the_same(shapefile, rasterfile, year):
    # Read shape file using gpd
    shape_file = gpd.read_file(shapefile)

    # Read imagery file 
    raster_file = rasterio.open(rasterfile)
    
    # Check coordinate reference system (CRS) of both datasets
    print('Raster file Projection: ', raster_file.crs)
    
    # Transform projection of shapefile to the raster file's coordinate system
    # Specify output projection system
    dst_crs = 'ESRI:102039'
    
    shape_file = shape_file.to_crs(dst_crs)
    shape_file.to_file('output/reprojected_shape/reprojected_sierra.shp')

In [26]:
make_projections_the_same('Sierra_Nevada_Conservancy_Boundary/Sierra_Nevada_Conservancy_Boundary.shp', 'input/ravg_2021_cbi4.tif', 2021)

Raster file Projection:  ESRI:102039


/Users/schen/.pyenv/versions/3.8.12/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [27]:
def clip_raster_with_shapefile(shapefile, rasterfile, year, region):
    # Read Shape file
    with fiona.open(shapefile, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    # read raster file
    with rasterio.open(rasterfile) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

    # Save clipped raster
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open("output/clipped_tif/"+str(year)+"_"+region+"_clipped.tif", "w", **out_meta) as dest:
        dest.write(out_image)

In [28]:
def read_clipped_tiff(file):
    df = gdal.Open(file)
    # Get data from raster with classifications
    band = df.GetRasterBand(1)
    arr = band.ReadAsArray()
    return arr, df

In [29]:
def extract_one_class(df, arr, classname, sel_area_name, year):
    sel_arr = np.where(arr == classname, 1, 0)
    # calculate the number of cells in the selected class
    sel_arr_ncells = np.sum(arr == classname)
    # based on this tutorial: https://gdal.org/tutorials/geotransforms_tut.html
    # extract the resolution of the data set
    dx = df.GetGeoTransform()[1]
    dy = -df.GetGeoTransform()[5]
    # The area of a cell can be calculated by multiplying the x- and y-resolution
    area_size = sel_arr_ncells * dx * dy
#     print("Total area of the fire burnt at the " + sel_area_name + " in " + str(year) + " was", area_size)
    return sel_arr, area_size

In [30]:
def write_geotiff(file, arr, input_df):
    if arr.dtype == np.float32:
        arr_type = gdal.GDT_Float32
    else:
        arr_type = gdal.GDT_Int32
    
    driver = gdal.GetDriverByName("GTiff")
    output_df = driver.Create(file, arr.shape[1], arr.shape[0], 1, arr_type)
    output_df.SetProjection(input_df.GetProjection())
    output_df.SetGeoTransform(input_df.GetGeoTransform())
    band = output_df.GetRasterBand(1)
    band.WriteArray(arr)
    band.FlushCache()
    band.ComputeStatistics(False)

In [31]:
burn_index_dict = {
    
    1: 'unchanged',
    2: 'low',
    3: 'medium',
    4: 'high'
    
}

final = []

def create_df_to_compare_years(start_year, end_year, clipped_shapefile_path, region):
    
    for i in range((end_year + 1 - start_year)):
        year = start_year+i
        clip_raster_with_shapefile(clipped_shapefile_path, 'input/ravg_'+str(year)+'_cbi4.tif', year, region)
        arr, df = read_clipped_tiff("output/clipped_tif/"+str(year)+"_"+ region +"_clipped.tif")
        
        for n in range(4):
            sel_index = n+1
            sel_area_name = burn_index_dict[sel_index]
            sel_arr, sel_area_size = extract_one_class(df, arr, sel_index, sel_area_name, year)
            final.append({
                'year': year,
                'severity_index': sel_index,
                'severity': sel_area_name,
                'sqm': sel_area_size
            })
            write_geotiff("output/tif_file_by_index/"+str(year)+"_"+region+"_clipped_"+sel_area_name+".tif", sel_arr, df)
            print(str(year) + sel_area_name + ' finished!')

In [32]:
create_df_to_compare_years(2012, 2021, 'output/reprojected_shape/reprojected_sierra.shp', 'sierra')

2012unchanged finished!
2012low finished!
2012medium finished!
2012high finished!
2013unchanged finished!
2013low finished!
2013medium finished!
2013high finished!
2014unchanged finished!
2014low finished!
2014medium finished!
2014high finished!
2015unchanged finished!
2015low finished!
2015medium finished!
2015high finished!
2016unchanged finished!
2016low finished!
2016medium finished!
2016high finished!
2017unchanged finished!
2017low finished!
2017medium finished!
2017high finished!
2018unchanged finished!
2018low finished!
2018medium finished!
2018high finished!
2019unchanged finished!
2019low finished!
2019medium finished!
2019high finished!
2020unchanged finished!
2020low finished!
2020medium finished!
2020high finished!
2021unchanged finished!
2021low finished!
2021medium finished!
2021high finished!


In [33]:
final_df = pd.DataFrame(final)

In [36]:
final_df.to_csv('output/sum_of_sierra_fire_size_by_burn_index.csv')

# Making sure the numbers are correct

## 1. Using CalFire data (x)
I tried comparing our analysis with Calfire's data, which documents [acres burned by year.](https://www.fire.ca.gov/stats-events/)

The numbers don't match bc CBI-4 data has records for only wildland fires reported within the conterminous United States (CONUS) that include at least 1000 acres of forested National Forest System (NFS) land (500 acres for Regions 8 and 9 as of 2016).

## 2. QGIS Grass r.report (v)
I also tried using QGIS to do the same cleaning and analysis with the help of the GRASS plugin.
https://grass.osgeo.org/grass78/manuals/r.report.html

Clean and anlyze the geodata all in QGIS and compare the results. The steps are:
1. Crop the tifs with Sierra Nevada's vector file
2. Use Grass r.report to find out the areas burned by index (in sq. meter)

The final results calculated by the GRASS plugin, stored in the "qgis-grass" folder, are the same as the results produced by my analysis. 

## 3. Areas burned by a single wildfire (~v)

The third way to verify our analysis result is to get the size of a single wildfire using our python script and compare the number with the state [official's data.](https://hub.arcgis.com/maps/CALFIRE-Forestry::california-fire-perimeters/about) 

In [55]:
# calculate acres burned by the North Complex Fire in 2020
final = []
create_df_to_compare_years(2020, 2020, 'output/reprojected_shape/north_complex_fire.shp', 'nc')
final_df = pd.DataFrame(final)
final_df.to_csv('output/sum_of_nc_fire_size_by_burn_index.csv')

2020unchanged finished!
2020low finished!
2020medium finished!
2020high finished!


In [56]:
# calculate acres burned by the Creek Fire in 2020
final = []
create_df_to_compare_years(2020, 2020, 'output/reprojected_shape/creek_fire.shp', 'crk')
final_df = pd.DataFrame(final)
final_df.to_csv('output/sum_of_crk_fire_size_by_burn_index.csv')

2020unchanged finished!
2020low finished!
2020medium finished!
2020high finished!


In [57]:
nc =  pd.read_csv('output/sum_of_nc_fire_size_by_burn_index.csv', index_col=0)
crk = pd.read_csv('output/sum_of_crk_fire_size_by_burn_index.csv', index_col=0)


# sq. meter to acre
nc['acre'] = nc.sqm.apply(lambda x: x*0.000247105)
crk['acre'] = crk.sqm.apply(lambda x: x*0.000247105)

### Data from U.S. Forest Service
- North Complex fire acre burned: [318,935](https://inciweb.nwcg.gov/incident/6997/) 
- Creek fire acre burned: [379,895](https://inciweb.nwcg.gov/incident/7147)

### By our script

In [58]:
# North Complex fire size
nc.acre.sum()

316604.14611750003

In [59]:
# Creek fire size
crk.acre.sum()

364200.5722185